# Note: The following file will fail if you will try to run it as it must contain private Spotify client settings.

First we had to install spotipy library by running `pip install spotipy`
<br>
According to https://anaconda.org/jkroes/spotipy

#### Using Spotify API request to approve few terms of use:<br>
* I understand that this app is not for commercial use
* I understand that I cannot migrate my app from non-commercial to commercial without permission
* I understand and agree with Spotify's Developer Terms of Service, Branding Guidelines, and Privacy Policy
    * https://developer.spotify.com/terms/
    * https://developer.spotify.com/branding-guidelines/
    * https://www.spotify.com/il-en/legal/privacy-policy/

Create access token and get a specific user liked songs (as a test)

In [41]:
import spotipy
import spotipy.util as util
import sys

username = ''
client_id = ''
client_secret = ''
redirect_uri = 'http://localhost:8888/callback/'
scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_saved_tracks()
    for item in results['items']:
        track = item['track']
        print(track['name'] + ' - ' + track['artists'][0]['name'])
else:
    print("Can't get token for", username)

NI BIEN NI MAL - Bad Bunny
One Level Down - Original mix - Sphera
Backseat Freestyle - Kendrick Lamar
Tusa - KAROL G
Ready To Let Go - Cage The Elephant
Diggin' a Hole - Downstairs Monsters
Starry Night - Original Mix - Peggy Gou
That's Life (feat. Mac Miller & Sia) - 88-Keys
Eleven - Khalid
Lalala - Y2K
מאושרים - Doli & Penn
ROXANNE - Arizona Zervas
Be Still - Liam Gallagher
לא חסר לי כלום - Avihu Pinchasov Rhythm Club
Sunday Best - Surfaces
Right Back (feat. A Boogie Wit Da Hoodie) - Khalid
I Got A Name - Stereo Version - Jim Croce
Lover Of The Light - Live From Red Rocks, Colorado - Mumford & Sons
What's the Use? - Mac Miller
Hard Sun - Eddie Vedder


In [82]:
track_id = []

if token:
    sp = spotipy.Spotify(auth=token)
    playlists = sp.user_playlists(username)
    print(playlists)
else:
    print("Can't get token for", username)

{'href': 'https://api.spotify.com/v1/users/-f/playlists?offset=0&limit=50', 'items': [], 'limit': 50, 'next': None, 'offset': 0, 'previous': None, 'total': 0}


##### We had the option to include 'popularity' column for each song but as the popularity is updated ongoing data then an old song current popularity isn't relevant as we wishing to have his popularity according the same year he has been chosen - we marked those lines.

* offset: The index of the first result to return. if you want to get the results with the start index 10 you will need to set the offset to 10.<br>
* limit: Maximum number of results to return.

Get audio features for songs who doesn't include in 'Billboard Year-End Hot 100 singles' (if a song that picked up will be part of the list, then he will be removed)<br>
Get 900 songs: 3 years, 300 each year.

In [44]:
songs_name_no={"2016": [],"2017": [], "2018":[]}
artists_name_no={"2016": [],"2017": [], "2018":[]}
track_id_no={"2016": [],"2017": [], "2018":[]}

for i in range(6,9): 
    for j in range(0,300,2):
        # spotify API for python doesn't work with multiple concatenation and NOT value
        # so had to take all the songs and eliminate the hebrew one later
        track_results = sp.search(q='year:201{}'.format(i), type='track', limit=2,offset=j)
        
        for j, k in enumerate(track_results['tracks']['items']):
            #print("******************************{}".format(i))
            songs_name_no["201{}".format(i)].append(k['name'])
            artists_name_no["201{}".format(i)].append(k['artists'][0]['name'])
            track_id_no["201{}".format(i)].append(k['id'])

    print('Number of elements in 201{}_track_id list:'.format(i), len(track_id_no["201{}".format(i)]))

Number of elements in 2016 track_id list: 300
Number of elements in 2017 track_id list: 300
Number of elements in 2018 track_id list: 300


In [64]:
import pandas as pd
import numpy as np

In [78]:
songs_df_no={"2016_df_no": pd.DataFrame(),"2017_df_no": pd.DataFrame(), "2018_df_no":pd.DataFrame()}

i = 6
for key, value in songs_df_no.items():
    songs_df_no[key]['Title'] = songs_name_no["201{}".format(i)]
    songs_df_no[key]['Artist'] = artists_name_no["201{}".format(i)]
    songs_df_no[key]['id'] = track_id_no["201{}".format(i)]
    songs_df_no[key]['Year'] = "201{}".format(i)
    i = i + 1
songs_df_no['2018_df_no'].head()

,Title,Artist,id,Year
0,Falling,Trevor Daniel,4TnjEaWOeW0eKTKIEvJyCa,2018
1,לשוב הביתה,Ishay Ribo,52n4gF126eIllrGuc9Zus6,2018
2,Lucid Dreams,Juice WRLD,285pBltuF7vW8TeWk8hdRR,2018
3,לבחור נכון,Amir Dadon,7n6emXIcaECmkljP1rPlvQ,2018
4,אחת ולתמיד,Ishay Ribo,3bgNXXL7TjlBDOl36wLWHk,2018


Concatenate between the 3 dataframes ignoring the indexes as all of them use indexes 0-299.

In [125]:
df_spotipy_no = pd.concat([songs_df_no['2016_df_no'], songs_df_no['2017_df_no'], songs_df_no['2018_df_no']],axis=0, sort=False, ignore_index=True)
mid = len(df_spotipy_no)/2
df_spotipy_no.iloc[np.r_[0:2, mid:mid+2, -2:0]]

,Title,Artist,id,Year
0,אמא אם הייתי,Hanan Ben Ari,6GeJojYQJDdKpsB71XawyK,2016
1,בדיוק כמו הירח,Yuval Dayan,4oUdKVvn9QpGfYOHfErtLz,2016
450,Barking,Ramz,2U5cq89GCnsR1yixKkC8d5,2017
451,היי,Elai Botner,5pzwB4XbcdYYsAuv12dNtw,2017
898,NO BYSTANDERS,Travis Scott,6OaVWaGfhXn70ZJfdVcabn,2018
899,Let It Be Me (feat. Ava Max),David Guetta,2F83FxNVkK6PPMHuYnwyVc,2018


* Eliminate songs in Hebrew in order to be more relevant to top100 list.<br>
Also our lyrics site doesn't include Hebrew songs.
* Including Hebrew songs that written in English letters, we had to include because there is no way to determine their origin<br>
#### The reasons above forced us to take much bigger songs in the first query as the elimination reduced it dramatically. 

In [127]:
df_spotipy_no = df_spotipy_no[~df_spotipy_no['Title'].str.contains('[א-ת]', regex = True)]
print(len(df_spotipy_no))
df_spotipy_no.iloc[np.r_[0:2, mid:mid+2, -2:0]]

654


,Title,Artist,id,Year
2,goosebumps,Travis Scott,6gBFPUFcJLzWGx4lenP6h2,2016
6,Say You Won't Let Go,James Arthur,0p6RzKrGeXzyYYd2RZPKd8,2016
607,Shallow,Lady Gaga,2VxeLyX666F8uXCJ0dZF8B,2018
608,Hope,XXXTENTACION,2ZRo7axmMPeSVUvDbGkJah,2018
898,NO BYSTANDERS,Travis Scott,6OaVWaGfhXn70ZJfdVcabn,2018
899,Let It Be Me (feat. Ava Max),David Guetta,2F83FxNVkK6PPMHuYnwyVc,2018


In [79]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
from pprint import pprint

pl2015_uri = 'spotify:playlist:1xkuRzFhnX3hCWu6hLxJ0U' #contains 95
pl2016_uri = 'spotify:playlist:2LWafCgWzsXGWv7wJeePjA' #contains 98
pl2017_uri = 'spotify:playlist:3CWVCLSmoT7speb9E5b9st' #contains 100
pl2018_uri = 'spotify:playlist:37IRJrV9jd0LnsFTIY83ax' #contains 100

ids2015_list = []
ids2016_list = []
ids2017_list = []
ids2018_list = []

offset = 0
i = 0

while True:
    tracks_id = sp.playlist_tracks(pl2018_uri,
                                  offset=offset,
                                  fields='items.track.id,total')
    songs_name = sp.playlist_tracks(pl2018_uri,
                                  offset=offset,
                                  fields='items.track.name.total')
    artist_name = sp.playlist_tracks(pl2018_uri,
                                  offset=offset,
                                  fields='items.track.artists.name.total')
#     popularity = sp.playlist_tracks(pl2018_uri,
#                                   offset=offset,
#                                   fields='items.track.popularity.total')
    

    offset = offset + len(tracks_id['items'])
    print(offset)
    print(len(tracks_id['items']))
    #print(offset, "/", response['total'])
    
    for i in range(0, offset - 1):
        print(tracks_id['items'][i])
        print(tracks_id['items'][i].get('track').get('id'))
        print()
        print(songs_name['items'][i])
        print(songs_name['items'][i].get('track').get('name'))
        print()
        print(artist_name['items'][i])
        print(artist_name['items'][i].get('track').get('artists')[0].get('name'))
        print(i)
        #pprint(ids['items'][i].get('track').get('id'))
        #ids2018_list.append(response['items'][i].get('track').get('id'))
        print("########################################################")
        #pprint(pops.get('items')[i].get('track').get('popularity'))
    if len(ids['items']) == 0:
        break
#ids2018_list
    
    
#     print(len(tracks_id['items']))
#     offset = offset + len(tracks_id['items'])
#     print(offset)
#     print(len(tracks_id['items']))
    #print(offset, "/", response['total'])
    
#     for i in range(0, offset):
#         print(artist_name['items'][i].get('track'))
#         print(tracks_id['items'][i])
#         print(i)
#         #pprint(tracks_id['items'][i].get('track').get('id'))
#         #ids2018_list.append(response['items'][i].get('track').get('id'))
        
#         #pprint(pops.get('items')[i].get('track').get('popularity'))
#     if len(tracks_id['items']) == 0:
#         break
# #ids2018_list

100
100
{'track': {'id': '6DCZcSspjsKoFjzjrWoCdn'}}
6DCZcSspjsKoFjzjrWoCdn

{'track': {'name': "God's Plan"}}
God's Plan

{'track': {'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'}, 'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4', 'id': '3TVXtAsR1Inumwj472S9r4', 'name': 'Drake', 'type': 'artist', 'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}]}}
Drake
0
########################################################
{'track': {'id': '0tgVpDi06FyKpA1z0VMD4v'}}
0tgVpDi06FyKpA1z0VMD4v

{'track': {'name': 'Perfect'}}
Perfect

{'track': {'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6eUKZXaKkcviH0Ku9w2n3V'}, 'href': 'https://api.spotify.com/v1/artists/6eUKZXaKkcviH0Ku9w2n3V', 'id': '6eUKZXaKkcviH0Ku9w2n3V', 'name': 'Ed Sheeran', 'type': 'artist', 'uri': 'spotify:artist:6eUKZXaKkcviH0Ku9w2n3V'}]}}
Ed Sheeran
1
########################################################
{'track': {'id': '7iDa6hUg2VgEL1

In [22]:
feature_results = sp.audio_features('0pqnGHJpmpxLKifKRmU6WP')
print(feature_results)

[{'danceability': 0.776, 'energy': 0.78, 'key': 10, 'loudness': -4.374, 'mode': 0, 'speechiness': 0.128, 'acousticness': 0.0622, 'instrumentalness': 0, 'liveness': 0.081, 'valence': 0.666, 'tempo': 124.949, 'type': 'audio_features', 'id': '0pqnGHJpmpxLKifKRmU6WP', 'uri': 'spotify:track:0pqnGHJpmpxLKifKRmU6WP', 'track_href': 'https://api.spotify.com/v1/tracks/0pqnGHJpmpxLKifKRmU6WP', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0pqnGHJpmpxLKifKRmU6WP', 'duration_ms': 204347, 'time_signature': 4}]


In [23]:
import pandas as pd
temp_ids = ['0pqnGHJpmpxLKifKRmU6WP', '4gB7HrYHbJVJ5RFOjxmoq4', '76cy1WJvNGJTj78UqeA5zr']

temp_df = pd.DataFrame()
temp_df['ids'] = temp_ids
print(temp_df, '\n')

                      ids
0  0pqnGHJpmpxLKifKRmU6WP
1  4gB7HrYHbJVJ5RFOjxmoq4
2  76cy1WJvNGJTj78UqeA5zr 



In [24]:
# again measuring the time
import timeit
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(temp_df['ids']),batchsize):
    batch = temp_df['ids'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)

print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 0.08773110000038287


In [25]:
print('number of elements in the track_id list:', len(rows))
print('\n', rows[0])

number of elements in the track_id list: 3

 {'danceability': 0.776, 'energy': 0.78, 'key': 10, 'loudness': -4.374, 'mode': 0, 'speechiness': 0.128, 'acousticness': 0.0622, 'instrumentalness': 0, 'liveness': 0.081, 'valence': 0.666, 'tempo': 124.949, 'type': 'audio_features', 'id': '0pqnGHJpmpxLKifKRmU6WP', 'uri': 'spotify:track:0pqnGHJpmpxLKifKRmU6WP', 'track_href': 'https://api.spotify.com/v1/tracks/0pqnGHJpmpxLKifKRmU6WP', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0pqnGHJpmpxLKifKRmU6WP', 'duration_ms': 204347, 'time_signature': 4}


Load the audio features data into a dataframe

In [27]:
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (3, 18)


,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.06220,https://api.spotify.com/v1/audio-analysis/0pqn...,0.776,204347,0.780,0pqnGHJpmpxLKifKRmU6WP,0,10,0.0810,-4.374,0,0.1280,124.949,4,https://api.spotify.com/v1/tracks/0pqnGHJpmpxL...,audio_features,spotify:track:0pqnGHJpmpxLKifKRmU6WP,0.666
1,0.00585,https://api.spotify.com/v1/audio-analysis/4gB7...,0.544,221800,0.781,4gB7HrYHbJVJ5RFOjxmoq4,0,6,0.1910,-6.160,0,0.0311,144.075,4,https://api.spotify.com/v1/tracks/4gB7HrYHbJVJ...,audio_features,spotify:track:4gB7HrYHbJVJ5RFOjxmoq4,0.527
2,0.04030,https://api.spotify.com/v1/audio-analysis/76cy...,0.836,217947,0.544,76cy1WJvNGJTj78UqeA5zr,0,7,0.0824,-5.975,1,0.0943,97.028,4,https://api.spotify.com/v1/tracks/76cy1WJvNGJT...,audio_features,spotify:track:76cy1WJvNGJTj78UqeA5zr,0.510


In [222]:
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.dtypes

Shape of the dataset: (3, 18)


acousticness        float64
analysis_url         object
danceability        float64
duration_ms           int64
energy              float64
id                   object
instrumentalness      int64
key                   int64
liveness            float64
loudness            float64
mode                  int64
speechiness         float64
tempo               float64
time_signature        int64
track_href           object
type                 object
uri                  object
valence             float64
dtype: object

Processing the data - drop uneeded columns

In [223]:
df_audio_features.drop(['analysis_url', 'track_href', 'type', 'uri'], axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.dtypes

Shape of the dataset: (3, 14)


acousticness        float64
danceability        float64
duration_ms           int64
energy              float64
track_id             object
instrumentalness      int64
key                   int64
liveness            float64
loudness            float64
mode                  int64
speechiness         float64
tempo               float64
time_signature        int64
valence             float64
dtype: object

Merge between audio features dataframe and our original dataframe

In [ ]:
# the 'inner' method will make sure that we only keep track IDs present in both datasets
# df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
# print("Shape of the dataset:", df_audio_features.shape)
# df.head()

Cheching if we have any duplication in track

In [224]:
#df[df.duplicated(subset=['artist_name','track_name'],keep=False)]

Saving the new dataframe as csv for outer use

In [ ]:
#df.to_csv('myname.csv')